## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import seaborn as sns
import feature_selection as fs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
import prep_utils as pu
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pointbiserialr
from sklearn.feature_selection import RFE
import warnings

warnings.filterwarnings("ignore")
PAST_YEARS = 2
TEST_YEAR = 10
KAGGLE_TEST_YEAR = 11

### **Database Connection Setup**
The initial intake of data was loaded into a SQLite3 database. Further, in order to improve the effectiveness of further data transformation procedures, a calculated choice was taken to convert the data into a Pandas dataframe. Furthermore, Year 11 competition data has been added to the database, augmenting it.


In [ ]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

### **Data Preparation**

#### **Preparing Teams Dataframe**

+ Removed features deemed not important such as: arena, lgID, divID.
+ Removed features that were irrelevant (always have the same value for all entries)
+ Replaced features of the format feature_made & feature_attempted by its success rate; (3points, fieldgoal, regular_winrate, playoff_winrate)
+ Replaced FirstRound, Semis & Final by the teams playoff rank

In [ ]:
df_new_teams = pu.prepare_teams(df_teams,df_teams_post, PAST_YEARS)
df_new_teams = fs.fs_teams(df_new_teams)
df_new_teams = pu.playoff_rank(df_new_teams,df_teams, PAST_YEARS)
#df_team_results = df_new_teams[["year","tmID","confID","playoff","rank","team_playoffs_count","playoff_rank"]]
df_team_results = df_new_teams[["year","tmID","confID","playoff","rank","team_playoffs_count","playoff_rank","Winrate","PO_Winrate"]]

#### **Preparing Coaches Dataframe**

+ Replaced Wins/Losses by winrate. For both regular season & playoff
+ Created Coach Award Amount Feature
+ Created Coach PlayOff Appearances Feature

In [ ]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,PAST_YEARS)
df_new_coaches = pu.group_coaches(df_new_coaches)

df_new_coaches.drop("coachID", axis = 1, inplace = True)

df_final_coaches = df_new_coaches.copy()
df_final_coaches.columns = df_final_coaches.columns.str.lower()


#### **Preparing Players Dataframe**
+ Created Player Award Amount Feature
+ Replaced features of the format feature_made & feature_attempted by its success rate;
+ In order to reduce the dataset dimensionality, we also aggregated the metrics that were common to both Regular Season & Playoffs in a **weighted** manner.


In [ ]:
df_new_players_teams = pu.prepare_player_teams(df_players_teams,df_awards,PAST_YEARS)

#### **Created 2 Team Ratings:**
+  One rated the players performance from the previous seasons. The goal is to assess the roster's capacity for consistent performance across a range of seasons, player growth, and the team's general quality and stability.

+ A rating that evaluates whether the team has had a roster of talented players in previous years. The goal of this rating is to assess the team's historical trend of acquiring or keeping high-caliber players as well as to provide a quantitative measure of the team's general quality and stability.

In [ ]:
# How the team performed in the previous years
previous_team_ratings, df_new_players_team = pu.final_team_ratings(df_players_teams,df_awards, df_players, df_teams, PAST_YEARS)

# How the players performed in the previous years
previous_team_player_ratings = pu.final_player_team_ratings(df_teams, df_new_players_team, df_players, PAST_YEARS,df_new_players_teams[df_new_players_teams['year'] == KAGGLE_TEST_YEAR])

#### **Merging all dataframes into one final dataframe**

Although initially we are given a relational schema, in order for the machine learning models work with our data we will be merging the relevant data into a single dataframe. This consolidation not only streamlines the data for machine learning models but also facilitates a comprehensive analysis of the interplay between various features on a global scale. By merging the data based on the relational schema's key relationships, we ensure that the connections and dependencies within the dataset are accurately represented. This unified approach enables us to uncover patterns and correlations that might have been obscured when the data was compartmentalized in separate dataframes. In essence, our goal is to create a cohesive foundation that allows for a more nuanced understanding of how different features influence each other, offering insights that were previously elusive due to the fragmented nature of the dataset.

In [ ]:
df_players = df_new_players_teams.copy()
df_players = fs.fs_players(df_players,0.2)
df_players = df_players[df_players['year'] != 1]


df_team_results.columns = df_team_results.columns.str.lower()
previous_team_player_ratings.columns = previous_team_player_ratings.columns.str.lower()


merged_data = pd.merge(df_players, df_team_results, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, df_final_coaches, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_ratings, on=['tmid', 'year'], how='left')
merged_data = pd.merge(merged_data, previous_team_player_ratings, on=['tmid', 'year'], how='left')


### **Feature Selection**
During the process of **feature engineering**, certain derived attributes demonstrated an inherent capacity to diminish the dataset's dimensionality, consolidating multiple attributes into singular entities. Noteworthy exemplars of this phenomenon are most of the ratios e.g winrate & metric_success rate.

In order to optimize the efficiency and relevance of the dataset, a strategic undertaking in feature selection was embarked upon. This involved a meticulous examination of the interrelationships between various features, facilitated by the utilization of a **correlation matrix**. The primary objective was to discern redundancy among features and subsequently identify candidates for removal.

Further refinement of the dataset was achieved through the employment of the **point biserial correlation test**, a discerning tool instrumental in isolating features exhibiting minimal correlation with the target variable. Features failing to meet the requisite correlation criteria were systematically excluded from consideration.

Subsequently, the Recursive Feature Elimination (RFE) methodology was invoked to conduct a discriminating assessment of the feature ensemble. Through this iterative process, RFE identified and retained features deemed most consequential and germane to the dataset's predictive performance.


#### **Correlation Matrix**
We'll use a correlation matrix to gauge the relationships between continuous attributes. This tool provides a quantitative measure of correlation, helping us identify patterns and dependencies. By assessing correlation coefficients, we can understand how variables interact, crucial for refining our machine learning models and making informed decisions in development.

In [ ]:
data1_10 = merged_data[merged_data['year'] != 11]
fs.correlation_matrix(data1_10, 0.75)

Initially, we planned to automatically delete one feature from pairs with good correlation (> 0.9). Upon deeper examination, this strategy turned out to be less "viable". A pair should be chosen for retention or discarding based on predetermined standards, taking into account the particular context of the dataset. Even when two qualities are highly correlated, their individual and combined effects on other aspects might differ. It becomes necessary to do a detailed analysis to make sure that our choices for data preparation take into account the complexities of the dataset in question and to prevent us from missing important information.


#### **Point Bisserial Correlation**
In order to evaluate the relationship between the **continuous** variables and the **binary** target (playoff), we will apply a statistical measure designed to account for these kinds of situations. This approach considers the connection between a continuous and a binary variable, offering important new information about that relationship. We can better understand how the continuous attributes affect the playoff result by doing this test to find patterns and linkages in the data. By using this analytical method, we want to better understand the underlying dynamics and determinants that determine whether a team makes the playoffs or not given its attributes.

In [ ]:
fs.bisserial_corr(data1_10)

The majority of attributes demonstrate a statistically significant correlation, save for **total_ft_pct** and **total_three_pct**. This anomaly may be attributed to their representation as percentages, since the correlation test predominantly accounts for linear relationships hence their values. Despite registering low correlation coefficients, these features remain pertinent, as their relevance is nuanced and contextually contingent, underscoring the importance of considering non-linear dynamics in data interpretation.

### **Algorithms and Model Training**
From the Exploratory Data Analysis we can easily conclude that our dataset despite not having many data itself (~ 10 WNBA Seasons) is pretty balanced (56% - 44%).

We'll use **StandardScaler** to scale the features. Scaling is an important preprocessing step that ensures all characteristics have a comparable scale and range. The characteristics are transformed by StandardScaler by removing the mean and dividing by the standard deviation, resulting in a distribution with a mean of 0 and a standard deviation of 1. This scaling strategy is very effective for algorithms based on distance, such as k-nearest neighbors and support vector machines.

We'll employ **Recursive Feature Elimination** (RFE) to identify the most influential features for each model. We'll set a minimum acceptable number of features, ensuring a balance between model complexity and performance. This systematic approach aims to enhance model interpretability and efficiency by retaining the most informative attributes while meeting our specified criteria.

To evaluate the performance of the different models a thorough evaluation will be conducted using a range of metrics. **Accuracy**, measuring the overall correctness of predictions, provides a broad perspective on model performance. **Recall**, emphasizing the ability to capture all relevant instances, is crucial for scenarios where identifying true positives is paramount. **F1-measure**, which combines precision and recall, offers a balanced assessment, especially valuable when there's a need to strike a harmony between false positives and false negatives. **AUC**, or Area Under the Curve, assesses the model's ability to distinguish between positive and negative instances across various thresholds, providing insights into its overall discriminatory power. Through the comprehensive analysis of these metrics, we aim to gain a nuanced understanding of each model's strengths and weaknesses.

We will apply **GridSearch** to fine tune each model and find the parameters that generate the best results.

To choose the optimal model for **Kaggle** submission, we'll rigorously test each one across years 2 to 10. By averaging the accuracies over this period, we aim to identify a model that not only performs well across different time points but also maintains consistent effectiveness.

Algorithms tested:
- **Random Forest**
- **Logistic Regression**
- **K Nearest Neighbours**
- **Gradient Boosting**
- **Support Vector Machines**


#### **Dividing the dataset in both train & test sets**


We'll transform the remaining non-numerical attributes into integers, a step favored by models for optimal performance with numerical inputs. Employing a label encoder, each distinct **teamID** & **confID** will be assigned a unique integer. Following this encoding, we'll partition the dataset into training and testing sets, laying the groundwork for effective model evaluation and prediction.

The train & test data is divided in the following format:
- The training data will be all years prior to the year we want to predict
- The test data will be the year we want to predict

In [ ]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

merged_data['tmid'] = label_encoder.fit_transform(merged_data['tmid'])
merged_data['confid'] = label_encoder.fit_transform(merged_data['confid'])

x = merged_data.drop('playoff', axis=1)
y = merged_data['playoff']

x = scaler.fit_transform(x)

x_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(0, TEST_YEAR - 1)]['playoff']

x_test = merged_data[merged_data['year'] == TEST_YEAR].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == TEST_YEAR]['playoff']

#### **RFE**
We will be running Recursive Feature Elimination (RFE) across various models, including **Random Forest**, **Logistic Regression**, **Support Vector Machine**, and **Gradient Boosting** to discern the optimal set of features for each. This involves systematically evaluating the performance of different feature subsets by testing varying numbers of features. Through the **RFE** process, we aim to identify not only the ideal number of features but also the specific attributes that contribute most significantly to the models' efficacy. By iteratively refining the feature selection, we seek to enhance the models' predictive power and streamline their performance with the most influential and relevant variables.

In [ ]:
print("Final Features:")
print(x_train.columns)
min_features = 20

rfe_classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    #'Support Vector Machine': SVC(random_state=42, kernel='linear'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
}
"""rfe_classifiers = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42,max_iter=10000),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}"""


classifiers_features = {}

total_features = len(x_train.columns)

best_model_info = {}


for model_name, model in rfe_classifiers.items():
    print(f"\033[1mModel: {model_name}\033[0m")
    
    results = []

    for i in range(min_features, total_features):
        rfe = RFE(model, n_features_to_select=i)
        rfe.fit(x_train, y_train)
        
        selected_features = set(x_train.columns[rfe.support_])
        selected_features.add("tmid")
        selected_features.add("year")
        selected_features = list(selected_features)

        model.fit(x_train[selected_features], y_train)

        accuracy = model.score(x_test[selected_features], y_test)

        results.append((selected_features, accuracy))
    

    # Sort the results based on accuracy in descending order
    results = sorted(results, key=lambda x: x[1], reverse=True)

    classifiers_features[model_name] = results[0][0]
    best_model_info[model_name] = {'features': results[0][0], 'accuracy': results[0][1], 'model': model}
    
    # Print the results
    for features, accuracy in results[:1]:
        print("Selected Features:", features)
        print("Accuracy:" + str(accuracy) + '\n')

best_model_name = max(best_model_info, key=lambda k: best_model_info[k]['accuracy'])
best_model = best_model_info[best_model_name]['model']
best_features = best_model_info[best_model_name]['features']

print(f"\033[1mBest Model: {best_model_name}\033[0m")
print(f"Best Features: {best_features}")
print(f"Accuracy: {best_model_info[best_model_name]['accuracy']}\n")

While applying a similar approach, we encountered a deviation when working with **K-Nearest Neighbors (KNN)**. Recursive Feature Elimination (RFE) wasn't conducive to the intricacies of KNN. To address this, we seamlessly pivoted to an alternative strategy, implementing **SelectKBest** for feature selection tailored to KNN's dynamics. 

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
results = []
knn = KNeighborsClassifier()
for i in range(min_features, total_features):
   

    selector = SelectKBest(score_func=mutual_info_classif, k=i)
    selector.fit(x_train, y_train)


    selected_features = set(x_train.columns[selector.get_support()])
    selected_features.add("tmid")
    selected_features.add("year")
    selected_features = list(selected_features)

    knn.fit(x_train[selected_features], y_train)

    accuracy = knn.score(x_test[selected_features], y_test)

    results.append((selected_features, accuracy))

results = sorted(results, key=lambda x: x[1], reverse=True)

classifiers_features["K-Nearest Neighbors"] = results[0][0]
rfe_classifiers["K-Nearest Neighbors"] = KNeighborsClassifier()
# Print the best 3 results
for features, accuracy in results[:1]:
    print("Selected Features:", features)
    print("Accuracy:" + str(accuracy) + '\n')



#### **Grid Search**
Grid search is a hyperparameter optimization technique employed in machine learning to systematically evaluate and compare a predefined set of hyperparameter combinations. It operates by exhaustively searching through the specified grid of hyperparameter values, enabling a comprehensive exploration of the parameter space. This method is particularly beneficial when fine-tuning models, as it automates the process of hyperparameter selection, ensuring a rigorous examination of potential configurations. By systematically evaluating various combinations, grid search aids in identifying the optimal hyperparameter set that maximizes the model's performance metrics, contributing to enhanced predictive accuracy and generalization.

In [ ]:
#best_params = fs.grid_search(classifiers_features,x_train,x_test,y_train,y_test)
#print(best_params)


best_params = {'Random Forest': {'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 50}, 
    'Logistic Regression': {'C': 10, 'fit_intercept': True, 'max_iter': 10000, 'penalty': 'l2', 'solver': 'liblinear'}, 
    'Gradient Boosting': {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 50}, 
    }


#### **Models Performance**

Choosing the model with the most promising results in the **tenth** year prompted us to scrutinize its performance across the entire temporal spectrum. We conducted a comprehensive assessment by testing the model on all available years and calculating the average accuracy. Recognizing that excelling in a single year may not be indicative of overall effectiveness, this method ensures a robust evaluation, considering the model's consistency and adaptability across diverse temporal contexts.

In [ ]:
accuracy_scores = []
auc_scores = []
best_model = best_model_info[best_model_name]['model']
best_features = best_model_info[best_model_name]['features']
#print(best_model)
probs = False
"""
for test_year in range(3, 11):  # Testing from year 2 to 10
    x_train = merged_data[merged_data['year'].between(1, test_year - 1)].drop('playoff', axis=1)
    y_train = merged_data[merged_data['year'].between(1, test_year - 1)]['playoff']
    
    x_test = merged_data[merged_data['year'] == test_year].drop('playoff', axis=1)
    y_test = merged_data[merged_data['year'] == test_year]['playoff']
    
    # Training the model
    best_model.fit(x_train[best_features], y_train)
    if(probs):
        # Testing the model
        probabilities = best_model.predict_proba(x_test[best_features])
        x_test['probabilities'] = probabilities.tolist()

        
        top_4_teams = (
            x_test.groupby('confid')
            .apply(lambda x: x.iloc[np.argsort(-np.array(x['probabilities'].tolist())[:, 0])][:4])
            .reset_index(drop=True)['tmid']
            .tolist()
        )

        # Create a list where 1 represents the team with one of the top 4 probabilities in their conference
        final_list = [0 if tmid in top_4_teams else 1 for tmid in x_test['tmid']]


        # Calculating accuracy
        accuracy = accuracy_score(y_test, final_list)
        
        accuracy_scores.append(accuracy)
    else:
        predictions = best_model.predict(x_test[best_features])
    
        # Calculating accuracy
        accuracy = accuracy_score(y_test, predictions)
        auc = roc_auc_score(y_test, predictions)
        accuracy_scores.append(accuracy)
        auc_scores.append(auc)
    print(predictions)
    print(f"Accuracy for testing year {test_year}: {accuracy}")

# Calculating average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
average_auc = sum(auc_scores) / len(auc_scores)
print(f"\nAverage Accuracy across all years: {average_accuracy}\n\n")
print(f"\nAverage AUC across all years: {average_auc}\n\n")
"""
    
for model_name, model in rfe_classifiers.items():
    print("\033[1m" + model_name + "\033[0m")
    accuracy_scores = []
    auc_scores = []
    
    for test_year in range(3, 11): 
        x_train = merged_data[merged_data['year'].between(1, test_year - 1)].drop('playoff', axis=1)
        y_train = merged_data[merged_data['year'].between(1, test_year - 1)]['playoff']
        
        x_test = merged_data[merged_data['year'] == test_year].drop('playoff', axis=1)
        y_test = merged_data[merged_data['year'] == test_year]['playoff']
        
        # Training the model
        model.fit(x_train[classifiers_features[model_name]], y_train)
        
        predictions = model.predict(x_test[classifiers_features[model_name]])
        
        # Calculating accuracy
        accuracy = accuracy_score(y_test, predictions)
        accuracy_scores.append(accuracy)
        auc = roc_auc_score(y_test, predictions)
        auc_scores.append(auc)
        
        #print(f"Accuracy for testing year {test_year}: {accuracy}")
        #print(f"AUC for testing year {test_year}: {auc}")

    # Calculating average accuracy
    average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    average_auc = sum(auc_scores) / len(auc_scores)
    print(f"Average \033[1mAccuracy\033[0m across all years: {average_accuracy}")
    print(f"Average \033[1mAUC\033[0m across all years: {average_auc}")
    print("\n")


In [ ]:

x_train = merged_data[merged_data['year'].between(1, 10)].drop('playoff', axis=1)
y_train = merged_data[merged_data['year'].between(1, 10)]['playoff']

x_test = merged_data[merged_data['year'] == 11].drop('playoff', axis=1)
y_test = merged_data[merged_data['year'] == 11]['playoff']

y_t = [1,0,0,1,1,0,1,1,1,1,0,1]


for model_name, model in rfe_classifiers.items():
    model.fit(x_train[classifiers_features[model_name]], y_train)
    predictions = model.predict(x_test[classifiers_features[model_name]])

    print("11 year Accuray:",model_name + "-",accuracy_score(y_t, predictions))
